## Health Claims Credibility Checker

Ananya Nimbalkar & Sai Nandini Peesapati

## 0. Install/Import Libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/My Drive/Transformers-Final-proj')


In [3]:
%cd /content/drive/My Drive/Transformers-Final-proj

/content


In [4]:
!pip install "accelerate>=0.26.0"
!pip uninstall transformers datasets -y
!pip install transformers datasets --upgrade
!pip install tf-keras

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 95, in resolve
    result = self._result = resolver.resolve(
                            ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 546, in resolve
    state = resolution.resolve(requirements, max_rounds=max_rounds)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [4]:
from transformers import TrainingArguments
print('Transformers OK!')

Transformers OK!


In [5]:
!python -c "from transformers import TrainingArguments; print('Transformers OK!')"

Transformers OK!


In [5]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
from datasets import Dataset, DatasetDict
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score


## 1. Load Dataset

We load the claims.csv file from the Monant Medical Misinformation dataset, which contains healthcare claims and their corresponding credibility ratings.

In [6]:
# Load dataset
claims_df = pd.read_csv("claims.csv")

## 2. Map Labels to Binary Classes

We convert multi-class labels like true, false, mixture, etc. into a binary classification task:
1 = credible claim, 0 = not credible claim. This simplifies the task for the model.

In [7]:
# Label mapping
def map_label(rating):
    if rating in ["true", "mostly-true"]:
        return 1
    elif rating in ["false", "mostly-false", "mixture", "unknown"]:
        return 0
    else:
        return None

## 3. Preprocess and Prepare Hugging Face Dataset

We drop any rows with missing labels or statements, split the data into training and testing sets, and convert them into Hugging Face DatasetDict format for use with the Trainer API.

In [8]:
claims_df["label"] = claims_df["rating"].map(map_label)
claims_df = claims_df.dropna(subset=["label", "statement"])

# Train-test split
train_df, test_df = train_test_split(claims_df[["statement", "label"]], test_size=0.2, random_state=42)

# Ensure labels are integers
train_df["label"] = train_df["label"].astype(int)
test_df["label"] = test_df["label"].astype(int)

# Convert to Huggingface Dataset
dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "test": Dataset.from_pandas(test_df)
})

## 4. Tokenize Text and Load BioBERT

We load the pre-trained BioBERT tokenizer and model, then tokenize the healthcare statements.
BioBERT is pre-trained on biomedical literature, making it well-suited for this task.

In [9]:
# Load tokenizer
model_checkpoint = "dmis-lab/biobert-base-cased-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Tokenization
def tokenize(example):
    return tokenizer(example["statement"], truncation=True)

tokenized_datasets = dataset.map(tokenize, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Load model
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

Map:   0%|          | 0/1895 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/474 [00:00<?, ? examples/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 5. Define Evaluation Metrics

We define accuracy, precision, recall, and F1 score to evaluate how well the model distinguishes between credible and non-credible claims.

In [10]:
# Metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds),
        "precision": precision_score(labels, preds),
        "recall": recall_score(labels, preds),
    }

## 6. Set Training Hyperparameters

We configure training parameters, such as batch size, number of epochs, evaluation strategy, and model saving behavior.

In [11]:
# Training args
training_args = TrainingArguments(
    gradient_accumulation_steps=4,
    output_dir="./biobert_misinformation",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=1,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## 7. Train the Model Using Hugging Face Trainer

We use Hugging Face’s Trainer API to train the model and handle evaluation, checkpointing, and logging automatically.

In [12]:
import torch
torch.cuda.empty_cache()

import os
os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"


In [13]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Train
trainer.train()

<ipython-input-13-81c83f533dcf>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 11199a186 (11199a186-vanderbilt-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.408500,0.443731,0.824895,0.278261,0.246154,0.320000
2,0.180500,0.348909,0.871308,0.207792,0.296296,0.160000


TrainOutput(global_step=177, training_loss=0.31061335876163115, metrics={'train_runtime': 106.5065, 'train_samples_per_second': 53.377, 'train_steps_per_second': 1.662, 'total_flos': 121465321484700.0, 'train_loss': 0.31061335876163115, 'epoch': 2.962025316455696})

## 8. Evaluate Model Performance

After training, we evaluate the model on the test set and print key performance metrics to assess its effectiveness.

In [14]:
# Evaluate
metrics = trainer.evaluate()
print(metrics)

{'eval_loss': 0.4437311887741089, 'eval_accuracy': 0.8248945147679325, 'eval_f1': 0.2782608695652174, 'eval_precision': 0.24615384615384617, 'eval_recall': 0.32, 'eval_runtime': 1.1123, 'eval_samples_per_second': 426.126, 'eval_steps_per_second': 53.94, 'epoch': 2.962025316455696}


## 9. Save the Fine-Tuned Model and Tokenizer

We save the fine-tuned model and tokenizer to disk so they can be reloaded for inference later (e.g., in the Streamlit app).

In [16]:
# Save model and tokenizer
model.save_pretrained("/content/drive/My Drive/Transformers-Final-proj/biobert_misinformation_model")
tokenizer.save_pretrained("/content/drive/My Drive/Transformers-Final-proj/biobert_misinformation_model")


('/content/drive/My Drive/Transformers-Final-proj/biobert_misinformation_model/tokenizer_config.json',
 '/content/drive/My Drive/Transformers-Final-proj/biobert_misinformation_model/special_tokens_map.json',
 '/content/drive/My Drive/Transformers-Final-proj/biobert_misinformation_model/vocab.txt',
 '/content/drive/My Drive/Transformers-Final-proj/biobert_misinformation_model/added_tokens.json',
 '/content/drive/My Drive/Transformers-Final-proj/biobert_misinformation_model/tokenizer.json')

In [18]:
# Prepare Explanation Dataset
import pandas as pd

claims_df = pd.read_csv("claims.csv")

# Only keep claims with a usable description (non-null, decently long)
explanation_df = claims_df[["statement", "description"]].dropna()
explanation_df = explanation_df[explanation_df["description"].str.len() > 30]

# Optional: combine rating into target to enrich generation
def create_target(row):
    return f"This claim is rated as '{row['rating']}'. {row['description']}"

explanation_df["target"] = claims_df.loc[explanation_df.index].apply(create_target, axis=1)

# Keep only statement + enriched target
final_explanation_data = explanation_df[["statement", "target"]]
final_explanation_data = final_explanation_data.dropna()

# Save to CSV
final_explanation_data.to_csv("/content/drive/My Drive/Transformers-Final-proj/claim_explanations.csv", index=False)
print("Saved claim_explanations.csv with", len(final_explanation_data), "rows.")


Saved claim_explanations.csv with 2608 rows.


# **Model inference**

The confidence score here represents the model's confidence in the claim being credible.

1. Eating dark chocolate can improve heart health.
2. Running regularly can reduce the risk of chronic diseases like diabetes.
3. Taking multivitamins improves overall health.
4. Yoga helps in reducing stress levels.
5. Consuming too much caffeine can lead to increased anxiety.
6. Drinking energy drinks improves physical performance.
7. Eating pineapple helps with weight loss.
8. Wearing glasses can improve eyesight.
9. Consuming garlic prevents colds.
10. Meditation can cure mental health disorders.
11. Using a standing desk can significantly reduce back pain.
12. Drinking lemon water detoxifies the body.

In [5]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the saved model and tokenizer
model_path = "/content/drive/My Drive/Transformers-Final-proj/biobert_misinformation_model"
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)


In [93]:
import torch
# Inference function for BioBERT to predict the credibility percentage
def predict_claim_credibility(claim):
    # Tokenize the input claim
    inputs = tokenizer(claim, return_tensors="pt", padding=True, truncation=True)

    # Get model output (logits)
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # Apply sigmoid to get probability (confidence score)
    probability = torch.sigmoid(logits)

    # Convert to percentage
    confidence_score = probability[0][1].item() * 100  # Confidence of the claim being credible
    return confidence_score


claim = "Wearing glasses can improve eyesight."
credibility_percentage = predict_claim_credibility(claim)
print(f"Claim: {claim}\nCredibility: {credibility_percentage}%")


Claim: Wearing glasses can improve eyesight.
Credibility: 52.63313055038452%


In [67]:
claim = "In 2020, two school boys in China died suddenly after wearing face masks during physical exercise."
credibility_percentage = predict_claim_credibility(claim)
print(f"Claim: {claim}\nCredibility: {credibility_percentage}%")

Claim: In 2020, two school boys in China died suddenly after wearing face masks during physical exercise.
Credibility: 68.1206464767456%


In [58]:
claim = "In January 2020, the Food and Drug Administration approved a nasal spray containing cocaine."
credibility_percentage = predict_claim_credibility(claim)
print(f"Claim: {claim}\nCredibility: {credibility_percentage}%")

Claim: In January 2020, the Food and Drug Administration approved a nasal spray containing cocaine.
Credibility: 67.90470480918884%


In [62]:
claim = "A viral social media post in January 2020 represented an authentic, accurate 'health bulletin' about the new coronavirus outbreak from an official public health authority."
credibility_percentage = predict_claim_credibility(claim)
print(f"Claim: {claim}\nCredibility: {credibility_percentage}%")

Claim: A viral social media post in January 2020 represented an authentic, accurate 'health bulletin' about the new coronavirus outbreak from an official public health authority.
Credibility: 65.17950892448425%


In [34]:
claim = "A photograph shows vintage box of fake snow decor made of the carcinogen asbestos."
credibility_percentage = predict_claim_credibility(claim)
print(f"Claim: {claim}\nCredibility: {credibility_percentage}%")

Claim: A photograph shows vintage box of fake snow decor made of the carcinogen asbestos.
Credibility: 64.91941213607788%


In [61]:
claim = "A Kentucky couple were placed under house arrest in July 2020 after a woman diagnosed with COVID-19 refused to agree to self-isolate because it would require her to get prior approval to go to the hospital."
credibility_percentage = predict_claim_credibility(claim)
print(f"Claim: {claim}\nCredibility: {credibility_percentage}%")

Claim: A Kentucky couple were placed under house arrest in July 2020 after a woman diagnosed with COVID-19 refused to agree to self-isolate because it would require her to get prior approval to go to the hospital.
Credibility: 64.34209942817688%


In [90]:
claim = "A video shows a genuine public service announcement from 1956 about how people can avoid a future plague."
credibility_percentage = predict_claim_credibility(claim)
print(f"Claim: {claim}\nCredibility: {credibility_percentage}%")

Claim: A video shows a genuine public service announcement from 1956 about how people can avoid a future plague.
Credibility: 63.763582706451416%


In [72]:
claim = "Born Basic Anti-Bac hand sanitizer was recalled in the U.S. after being found to contain methanol, a poisonous chemical."
credibility_percentage = predict_claim_credibility(claim)
print(f"Claim: {claim}\nCredibility: {credibility_percentage}%")

Claim: Born Basic Anti-Bac hand sanitizer was recalled in the U.S. after being found to contain methanol, a poisonous chemical.
Credibility: 63.451576232910156%


In [106]:
claim = "A doctor in Italy shared numerous details about how hospitals in the country are dealing with COVID-19, a disease caused by the new coronavirus."
credibility_percentage = predict_claim_credibility(claim)
print(f"Claim: {claim}\nCredibility: {credibility_percentage}%")

Claim: A doctor in Italy shared numerous details about how hospitals in the country are dealing with COVID-19, a disease caused by the new coronavirus.
Credibility: 63.25032711029053%


In [52]:
claim = "Carsyn Leigh Davis died of COVID-19 shortly after attending a 'COVID party' at her youth church."
credibility_percentage = predict_claim_credibility(claim)
print(f"Claim: {claim}\nCredibility: {credibility_percentage}%")

Claim: Carsyn Leigh Davis died of COVID-19 shortly after attending a 'COVID party' at her youth church.
Credibility: 61.904627084732056%


In [45]:
claim = "A group of 43 European countries and territories have far more people than the UK, but fewer Covid-19 deaths."
credibility_percentage = predict_claim_credibility(claim)
print(f"Claim: {claim}\nCredibility: {credibility_percentage}%")

Claim: A group of 43 European countries and territories have far more people than the UK, but fewer Covid-19 deaths.
Credibility: 61.181193590164185%


In [64]:
claim = "Health experts predicted the new coronavirus could kill 65 million people."
credibility_percentage = predict_claim_credibility(claim)
print(f"Claim: {claim}\nCredibility: {credibility_percentage}%")

Claim: Health experts predicted the new coronavirus could kill 65 million people.
Credibility: 59.04858708381653%


In [108]:
claim = "On average, do men and women differ cognitively?"
credibility_percentage = predict_claim_credibility(claim)
print(f"Claim: {claim}\nCredibility: {credibility_percentage}%")

Claim: On average, do men and women differ cognitively?
Credibility: 40.32866060733795%


In [107]:
claim = "The black lung lie: It’s the widespread belief that smokers’ lungs turn black."
credibility_percentage = predict_claim_credibility(claim)
print(f"Claim: {claim}\nCredibility: {credibility_percentage}%")

Claim: The black lung lie: It’s the widespread belief that smokers’ lungs turn black.
Credibility: 36.35234832763672%
